In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.cross_validation import StratifiedKFold
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GMM

def make_ellipses(gmm, ax):
    for n, color in enumerate('rgb'):
        v, w = np.linalg.eigh(gmm._get_covars()[n][:2, :2])
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v *= 9
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                  180 + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)

## randontly select small sample training dataset

In [4]:
train=pd.read_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/ADS Final Project/forem.csv',parse_dates=['srch_ci','date_time'])

## Drop useless columns

In [5]:
train= train.drop('Unnamed: 0',axis=1)

In [6]:
train['date_time']=pd.to_numeric(train['date_time'])

In [7]:
train['srch_ci']=pd.to_numeric(train['srch_ci'])

In [8]:
train_nhc=train.drop('hotel_cluster',axis=1)

In [9]:
train.head()

,date_time,site_name,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_ci,...,srch_rm_cnt,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,hotel_nights,adv_times
0,1407678330000000000,2,66,348,46035,NaN,1,1,1,1413676800000000000,...,1,1,0,2,4,96,201,65,4.0,-73.426736
1,1417538810000000000,2,66,337,44102,956.5488,0,0,9,1424304000000000000,...,1,1,0,1,2,50,682,71,1.0,-79.300810
2,1403035727000000000,2,66,174,26265,208.5084,0,0,9,1404604800000000000,...,1,1,1,1,2,50,628,79,2.0,-20.160567
3,1404916495000000000,24,3,50,22013,NaN,0,0,5,1405468800000000000,...,1,1,0,1,3,104,63,82,2.0,-8.392419
4,1415044643000000000,2,66,331,23846,412.8715,0,0,2,1419206400000000000,...,1,1,0,2,2,50,675,28,6.0,-54.168484


In [50]:
no_null_train_nhc = train_nhc.dropna()

In [49]:
test_nhc= train_nhc[train_nhc['orig_destination_distance'].isnull()]

In [51]:
X_train_nhc = no_null_train_nhc.drop('orig_destination_distance',axis=1)
y_train_nhc = no_null_train_nhc['orig_destination_distance']
X_test_nhc = test_nhc.drop('orig_destination_distance',axis=1)
y_test_nhc = test_nhc['orig_destination_distance']

In [90]:
X_test_nhc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269753 entries, 0 to 752451
Data columns (total 20 columns):
date_time                   269753 non-null int64
site_name                   269753 non-null int64
user_location_country       269753 non-null int64
user_location_region        269753 non-null int64
user_location_city          269753 non-null int64
is_mobile                   269753 non-null int64
is_package                  269753 non-null int64
channel                     269753 non-null int64
srch_ci                     269753 non-null int64
srch_adults_cnt             269753 non-null int64
srch_children_cnt           269753 non-null int64
srch_rm_cnt                 269753 non-null int64
srch_destination_type_id    269753 non-null int64
is_booking                  269753 non-null int64
cnt                         269753 non-null int64
hotel_continent             269753 non-null int64
hotel_country               269753 non-null int64
hotel_market                269753 non-

classifiers = dict((covar_type, GMM(n_components=n_classes,
                    covariance_type=covar_type, init_params='wc', n_iter=20))
                   for covar_type in ['spherical', 'diag', 'tied', 'full'])

n_classifiers = len(classifiers)

plt.figure(figsize=(3 * n_classifiers / 2, 6))
plt.subplots_adjust(bottom=.01, top=0.95, hspace=.15, wspace=.05,
                    left=.01, right=.99)


y_test_pred = classifier.predict(X_test[1])

## Build GMM model for imputation
### classify data into 20 group and calculate mean distance for each group

In [12]:
g1 = GMM(n_components=20)

In [52]:
g1.fit(X_train_nhc,y_train_nhc)

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=20, n_init=1, n_iter=100, params='wmc', random_state=None,
  thresh=None, tol=0.001, verbose=0)

In [53]:
y_pred_nhc=g1.predict(X_test_nhc)

In [59]:
y_train_pred_nhc = g1.predict(X_train_nhc)
y_train_pred_nhc

array([15, 16,  2, ..., 15, 10,  2])

In [60]:
no_null_train_nhc['y_predict_nhc'] = y_train_pred_nhc

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
test1=no_null_train_nhc[['y_predict_nhc','orig_destination_distance']]

In [62]:
test1=no_null_train_nhc[['y_predict_nhc','orig_destination_distance']].groupby(['y_predict_nhc']).mean()
test1.reset_index(level=['y_predict_nhc'], inplace = True)
test1

,y_predict_nhc,orig_destination_distance
0,0,1904.747874
1,1,1422.386060
2,2,987.418012
3,3,2762.222894
4,4,2337.632389
5,5,2467.905943
6,6,3743.522878
7,7,969.493322
8,8,2256.811251
9,9,2626.112550


In [65]:
g1.score(X_train_nhc,y_train_nhc)

array([ -94.96735477, -116.05609039, -101.53668172, ...,  -95.04853125,
       -119.11196509, -100.65915695])

## model on different dataset

In [66]:
g2 = GMM(n_components=20)
g2.fit(X_train_nhc)

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=20, n_init=1, n_iter=100, params='wmc', random_state=None,
  thresh=None, tol=0.001, verbose=0)

In [67]:
y_pred_nhc2=g2.predict(X_test_nhc)

In [68]:
y_train_pred_nhc2 = g2.predict(X_train_nhc)

In [69]:
no_null_train_nhc['y_predict_nhc2'] = y_train_pred_nhc2

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [77]:
g2.score(X_test_nhc)

array([-107.65424601, -122.84339837, -126.76502854, ..., -131.35436858,
       -101.82555056, -140.83848682])

In [81]:
test2=no_null_train_nhc[['y_predict_nhc2','orig_destination_distance']].groupby(['y_predict_nhc2']).mean()
test2.reset_index(level=['y_predict_nhc2'], inplace = True)
test2

,y_predict_nhc2,orig_destination_distance
0,0,4158.045993
1,1,2006.406523
2,2,765.905723
3,3,3229.125974
4,4,2805.630648
5,5,2457.086324
6,6,2045.687757
7,7,2259.714301
8,8,2256.469572
9,9,3323.407500


In [78]:
test_nhc['y_predict_nhc2'] = y_pred_nhc2

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [131]:
result = test_nhc.merge(test2, on='y_predict_nhc2', how='left', indicator=True)

In [132]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269753 entries, 0 to 269752
Data columns (total 24 columns):
date_time                      269753 non-null int64
site_name                      269753 non-null int64
user_location_country          269753 non-null int64
user_location_region           269753 non-null int64
user_location_city             269753 non-null int64
orig_destination_distance_x    0 non-null float64
is_mobile                      269753 non-null int64
is_package                     269753 non-null int64
channel                        269753 non-null int64
srch_ci                        269753 non-null int64
srch_adults_cnt                269753 non-null int64
srch_children_cnt              269753 non-null int64
srch_rm_cnt                    269753 non-null int64
srch_destination_type_id       269753 non-null int64
is_booking                     269753 non-null int64
cnt                            269753 non-null int64
hotel_continent                269753 non-nu

In [133]:
result=result.drop('orig_destination_distance_x',axis=1)

In [134]:
result.rename(columns={'orig_destination_distance_y': 'orig_destination_distance'}, inplace=True)

In [135]:
result=result.drop('_merge',axis=1)

In [176]:
final=pd.concat([no_null_train_nhc,result])

In [171]:
final=final.drop('y_predict_nhc',axis=1)

In [172]:
final=final.drop('y_predict_nhc2',axis=1)

In [173]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752455 entries, 1 to 269752
Data columns (total 21 columns):
adv_times                    752455 non-null float64
channel                      752455 non-null int64
cnt                          752455 non-null int64
date_time                    752455 non-null int64
hotel_continent              752455 non-null int64
hotel_country                752455 non-null int64
hotel_market                 752455 non-null int64
hotel_nights                 752455 non-null float64
is_booking                   752455 non-null int64
is_mobile                    752455 non-null int64
is_package                   752455 non-null int64
orig_destination_distance    752455 non-null float64
site_name                    752455 non-null int64
srch_adults_cnt              752455 non-null int64
srch_children_cnt            752455 non-null int64
srch_ci                      752455 non-null int64
srch_destination_type_id     752455 non-null int64
srch_rm_cnt     

In [174]:
final=final.reset_index(drop=True)

In [163]:
addhc = train[['date_time','hotel_cluster']]

In [167]:
final = final.merge(train, on='date_time', how='left', indicator=True)

In [121]:
final['date_time']=pd.to_datetime(final['date_time'])

In [122]:
final['srch_ci']=pd.to_datetime(final['srch_ci'])

In [125]:
final.to_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/ADS Final Project/finaltrain.csv',index=False)

In [182]:
X_train = train.dropna().drop('orig_destination_distance',axis=1)
X_test = train[train['orig_destination_distance'].isnull()].drop('orig_destination_distance',axis=1)

In [195]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269753 entries, 0 to 752451
Data columns (total 21 columns):
date_time                   269753 non-null int64
site_name                   269753 non-null int64
user_location_country       269753 non-null int64
user_location_region        269753 non-null int64
user_location_city          269753 non-null int64
is_mobile                   269753 non-null int64
is_package                  269753 non-null int64
channel                     269753 non-null int64
srch_ci                     269753 non-null int64
srch_adults_cnt             269753 non-null int64
srch_children_cnt           269753 non-null int64
srch_rm_cnt                 269753 non-null int64
srch_destination_type_id    269753 non-null int64
is_booking                  269753 non-null int64
cnt                         269753 non-null int64
hotel_continent             269753 non-null int64
hotel_country               269753 non-null int64
hotel_market                269753 non-

## different data input

In [184]:
g2.fit(X_train)

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=20, n_init=1, n_iter=100, params='wmc', random_state=None,
  thresh=None, tol=0.001, verbose=0)

In [188]:
no_null_train = train.dropna()
null_train = train[train['orig_destination_distance'].isnull()]

In [189]:
no_null_train['y_predict'] = g2.predict(X_train)

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## use this mean table as reference 

In [190]:
means=no_null_train[['y_predict','orig_destination_distance']].groupby(['y_predict']).mean()
means.reset_index(level=['y_predict'], inplace = True)
means

,y_predict,orig_destination_distance
0,0,2806.413908
1,1,2981.863080
2,2,3763.506324
3,3,2164.268154
4,4,2189.838912
5,5,2070.379825
6,6,4375.340809
7,7,2744.279203
8,8,3228.292094
9,9,3657.407685


In [191]:
null_train['y_predict'] = g2.predict(X_test)

/Users/Shuxian/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## merge back to the orginal dataset to fill null value

In [192]:
newresult = null_train.merge(means, on='y_predict', how='left', indicator=True)

In [196]:
newresult=newresult.drop('orig_destination_distance_x',axis=1)

In [197]:
newresult.rename(columns={'orig_destination_distance_y': 'orig_destination_distance'}, inplace=True)

In [198]:
newresult=newresult.drop('_merge',axis=1)

In [199]:
newfinal = pd.concat([no_null_train,newresult])

In [201]:
newfinal['date_time']= pd.to_datetime(newfinal['date_time'])

In [202]:
newfinal['srch_ci']= pd.to_datetime(newfinal['srch_ci'])

In [204]:
newfinal=newfinal.reset_index(drop=True)

In [205]:
newfinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752455 entries, 0 to 752454
Data columns (total 23 columns):
adv_times                    752455 non-null float64
channel                      752455 non-null int64
cnt                          752455 non-null int64
date_time                    752455 non-null datetime64[ns]
hotel_cluster                752455 non-null int64
hotel_continent              752455 non-null int64
hotel_country                752455 non-null int64
hotel_market                 752455 non-null int64
hotel_nights                 752455 non-null float64
is_booking                   752455 non-null int64
is_mobile                    752455 non-null int64
is_package                   752455 non-null int64
orig_destination_distance    752455 non-null float64
site_name                    752455 non-null int64
srch_adults_cnt              752455 non-null int64
srch_children_cnt            752455 non-null int64
srch_ci                      752455 non-null datetime64[ns

In [206]:
newfinal.to_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/ADS Final Project/newtrain.csv',index=False)

g3= GMM(n_components=100)

no_null_train = train.dropna()

null_train = train[train['orig_destination_distance'].isnull()]

X = no_null_train.drop('hotel_cluster',axis=1)
y=no_null_train['hotel_cluster']

g3.fit(X,y)

In [209]:
newfinal.columns.unique()

array(['adv_times', 'channel', 'cnt', 'date_time', 'hotel_cluster',
       'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_nights',
       'is_booking', 'is_mobile', 'is_package',
       'orig_destination_distance', 'site_name', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_ci', 'srch_destination_type_id',
       'srch_rm_cnt', 'user_location_city', 'user_location_country',
       'user_location_region', 'y_predict'], dtype=object)

## Delete outer liners

In [212]:
newfinal=newfinal[newfinal['adv_times']<1 ]

In [216]:
newfinal = newfinal[newfinal['hotel_nights']>= 0]

In [222]:
newfinal['adv_times']=abs(newfinal['adv_times'])

In [224]:
newfinal.to_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/ADS Final Project/newtrain.csv',index=False)

In [226]:
toneha=newfinal[newfinal['is_booking']==1]

In [227]:
toneha.to_csv('/Users/Shuxian/Documents/CSYE7390-Spring2016/ADS Final Project/toneha.csv',index=False)